### 已经拿到已经训练好的模型，可以在这里指定输入序列生成midi音乐

In [1]:
import pickle
import pretty_midi
import tensorflow as tf
import numpy as np

from tensorflow.keras import backend as K
from tqdm import tnrange, tqdm_notebook, tqdm
from numpy.random import choice
from keras_self_attention import SeqSelfAttention

d:\Code\Anaconda\envs\mytensorflowenv\lib\site-packages\requests\__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.0.0)/charset_normalizer (2.0.11) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
class NoteTokenizer:
    def __init__(self):
        self.notes_to_index = {}
        self.index_to_notes = {}
        self.num_of_word = 0
        self.unique_word = 0
        self.notes_freq = {}

    def transform(self, list_array):
        """Transform a list of note in string into index.

        Parameters
        ==========
        list_array : list
          list of note in string format

        Returns
        =======
        The transformed list in numpy array.

        """
        transformed_list = []
        for instance in list_array:
            transformed_list.append([self.notes_to_index[note] for note in instance])
        return np.array(transformed_list, dtype=np.int32)

    def partial_fit(self, notes):
        """Partial fit on the dictionary of the tokenizer

        Parameters
        ==========
        notes : list of notes

        """
        for note in notes:
            note_str = ",".join(str(a) for a in note)
            if note_str in self.notes_freq:
                self.notes_freq[note_str] += 1
                self.num_of_word += 1
            else:
                self.notes_freq[note_str] = 1
                self.unique_word += 1
                self.num_of_word += 1
                self.notes_to_index[note_str], self.index_to_notes[self.unique_word] = (
                    self.unique_word,
                    note_str,
                )

    def add_new_note(self, note):
        """Add a new note into the dictionary

        Parameters
        ==========
        note : str
          a new note who is not in dictionary.

        """
        assert note not in self.notes_to_index
        self.unique_word += 1
        self.notes_to_index[note], self.index_to_notes[self.unique_word] = (
            self.unique_word,
            note,
        )


In [3]:
def piano_roll_to_pretty_midi(piano_roll, fs=100, program=0):
    '''Convert a Piano Roll array into a PrettyMidi object
     with a single instrument.
    Parameters
    ----------
    piano_roll : np.ndarray, shape=(128,frames), dtype=int
        Piano roll of one instrument
    fs : int
        Sampling frequency of the columns, i.e. each column is spaced apart
        by ``1./fs`` seconds.
    program : int
        The program number of the instrument.
    Returns
    -------
    midi_object : pretty_midi.PrettyMIDI
        A pretty_midi.PrettyMIDI class instance describing
        the piano roll.
    '''
    notes, frames = piano_roll.shape
    pm = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=program)

    # pad 1 column of zeros so we can acknowledge inital and ending events
    piano_roll = np.pad(piano_roll, [(0, 0), (1, 1)], 'constant')

    # use changes in velocities to find note on / note off events
    velocity_changes = np.nonzero(np.diff(piano_roll).T)

    # keep track on velocities and note on times
    prev_velocities = np.zeros(notes, dtype=int)
    note_on_time = np.zeros(notes)

    for time, note in zip(*velocity_changes):
        # use time + 1 because of padding above
        velocity = piano_roll[note, time + 1]
        time = time / fs
        if velocity > 0:
            if prev_velocities[note] == 0:
                note_on_time[note] = time
                prev_velocities[note] = velocity
        else:
            pm_note = pretty_midi.Note(
                velocity=prev_velocities[note],
                pitch=note,
                start=note_on_time[note],
                end=time)
            instrument.notes.append(pm_note)
            prev_velocities[note] = 0
    pm.instruments.append(instrument)
    return pm

In [11]:
#在这里加载训练好的模型

model = tf.keras.models.load_model('./music_model/model_ep50.h5', custom_objects=SeqSelfAttention.get_custom_objects())
note_tokenizer  = pickle.load( open( "./music_model/tokenizer_ep50.p", "rb" ) )

In [14]:
def generate_from_ssvep_note(note_tokenizer, notes):
  generate = [note_tokenizer.notes_to_index['e'] for i in range(50-len(notes))]
  for i in range(len(notes)):
    generate += [note_tokenizer.notes_to_index[notes[i]]]
  return generate

In [15]:
def generate_notes(generate, model, unique_notes, max_generated=1000, seq_len=50):
  for i in tqdm_notebook(range(max_generated), desc='genrt'):
    test_input = np.array([generate])[:,i:i+seq_len]
    predicted_note = model.predict(test_input)
    random_note_pred = choice(unique_notes+1, 1, replace=False, p=predicted_note[0])
    generate.append(random_note_pred[0])
  return generate

In [16]:
noteToMidiId = {
    1: '60',
    2: '62',
    3: '64',
    4: '65',
    5: '67',
    6: '69',
    7: '71',
    'e': 'e',
    11: '72',
    77:'59'
}
# outputNoteList = [5,3,5,3,5,3,1,2,4,3,2,5,
#                   5,3,5,3,5,3,1,2,4,3,2,1,
#                   2,2,4,4,3,1,5,2,4,3,2,5,
#                   5,3,5,3,5,3,1,2,4,3,2,1]
outputNoteList = [5,3,5,3,5,3,1]
# outputNoteList = [1,1,5,5,6,6,5,4,4,3,3,2,2,1]
# outputNoteList = [1,1,5,5,6,6,5,'e','e','e','e',4,4,3,3,2,2,1,'e','e','e','e',5,5,4,4,3,3,2,'e','e','e','e',5,5,4,4,3,3,2,'e']
# outputNoteList = [5,3,5,11,6,11,6,5,5,1,2,3,2,1,2,5,3,5,11,7,6,11,5,5,2,3,4,77,1]
notes = [noteToMidiId[i] for i in outputNoteList]


# notes = ['67','64','67','64','67','64','60','e','62','65','64','62','67','e','67','64','67','64','67','64','60','e','62','65','64','62','60','e',]
generate = generate_from_ssvep_note(note_tokenizer, notes)

In [17]:
max_generate = 200
unique_notes = note_tokenizer.unique_word
seq_len=50
generate = generate_notes(generate, model, unique_notes, max_generate, seq_len)

d:\Code\Anaconda\envs\mytensorflowenv\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


genrt:   0%|          | 0/200 [00:00<?, ?it/s]

In [9]:
def write_midi_file_from_generated(generate, midi_file_name = "result.mid", start_index=49, fs=8, max_generated=1000):
  note_string = [note_tokenizer.index_to_notes[ind_note] for ind_note in generate]
  array_piano_roll = np.zeros((128,max_generated+1), dtype=np.int16)
  for index, note in enumerate(note_string[start_index:]):
    if note == 'e':
      pass
    else:
      splitted_note = note.split(',')
      for j in splitted_note:
        array_piano_roll[int(j),index] = 1
  generate_to_midi = piano_roll_to_pretty_midi(array_piano_roll, fs=fs)
  print("Tempo {}".format(generate_to_midi.estimate_tempo()))
  for note in generate_to_midi.instruments[0].notes:
    note.velocity = 100
  generate_to_midi.write(midi_file_name)

In [18]:
write_midi_file_from_generated(generate, "./midi/50.mid", start_index=49, fs=5, max_generated = 200)

Tempo 171.42857142857167


In [23]:
write_midi_file_from_generated(generate, "./midi/送别.mid", start_index=49, fs=5, max_generated = max_generate)

Tempo 176.4705882352946
